In [1]:
# Install the necessary libraries
!pip install transformers datasets torch

In [2]:
# Import required libraries
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForLanguageModeling
import torch

In [3]:
dataset = load_dataset("valurank/News_Articles_Categorization")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/857 [00:00<?, ?B/s]

news_article_categorization.csv:   0%|          | 0.00/19.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3722 [00:00<?, ? examples/s]

In [4]:
# Display a sample from the training dataset
sample = dataset["train"][0]  # Fetch the first data point
print("Sample Article:")
print(sample["Text"])  # Display the article
print("\nSample Summary (Highlights):")
print(sample["Category"])  # Display the summary

Sample Article:
Elon Musk, Amber Heard Something's Fishy On Wrapped-Up Sushi Last we heard, Elon Musk and Amber Heard were "not back together" even though they kiss goodbye and dance real close ... sorry, we're not buying that now. Amber and Elon went on a sushi date Monday in WeHo, and looked like the full-blown hand-holding couple that's definitely on again. But that's only because that's exactly what they are -- no matter how many times they try to say they're not reunited. We broke the story ... Elon and Amber started hanging out again this past fall ... after announcing their split in the summer. Since then, they've smooched and gone dancing together. If it looks like a reunited duck, walks like a reunited duck ... TMZ.com

Sample Summary (Highlights):
Entertainment


In [5]:
# Select a small subset for quick training and evaluation (to save time)
# Split the 'train' dataset into training and validation sets
train_test_split_dataset = dataset["train"].train_test_split(test_size=0.1) # 10% for validation

small_train_dataset = train_test_split_dataset["train"].select(range(100))
small_eval_dataset = train_test_split_dataset["test"].select(range(20))

In [6]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

In [8]:
#preprocess the dataset
def preprocess_function(examples):
    # Input: Full article to be summarized
    inputs = [text for text in examples["Text"]]
    # Target: Summary of the article (highlights)
    targets = [summary for summary in examples["Category"]]

    # Tokenize both the input and target text with truncation and padding
    model_inputs = tokenizer(inputs, max_length=256, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=64, truncation=True, padding="max_length")

    # Add the tokenized labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_train = small_train_dataset.map(preprocess_function, batched=True, remove_columns=small_train_dataset.column_names)
tokenized_eval = small_eval_dataset.map(preprocess_function, batched=True, remove_columns=small_eval_dataset.column_names)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [10]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [11]:
training_args = TrainingArguments(
    output_dir="./results",                # Directory to save model checkpoints and results
    do_eval=True,                         # Explicitly enable evaluation
    learning_rate=5e-5,                   # Initial learning rate for optimization
    per_device_train_batch_size=2,        # Number of samples per batch during training
    per_device_eval_batch_size=2,         # Number of samples per batch during evaluation
    num_train_epochs=10,                   # Train for only 1 epoch (for quick illustration)
    weight_decay=0.01,                    # Regularization to prevent overfitting
    save_strategy="epoch",                # Save and evaluate after each epoch
    save_total_limit=1,                   # Save only the latest model checkpoint
    logging_dir="./logs",                 # Directory to store training logs
    logging_steps=10,                     # Log every 10 steps
    report_to="none",                     # Disable logging to external platforms (e.g., WandB)
)

In [12]:
trainer = Trainer(
    model=model,                          # GPT-2 model
    args=training_args,                   # Training arguments
    train_dataset=tokenized_train,        # Training dataset
    eval_dataset=tokenized_eval,          # Evaluation dataset
    tokenizer=tokenizer,                  # Tokenizer to process input text
    data_collator=data_collator,          # Data collator for batching
)

/tmp/ipython-input-4231373555.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [13]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 50256}.
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.557300
20,3.617700
30,3.444100
40,3.357200
50,3.466500
60,2.942300
70,3.049000
80,3.013500
90,2.895800
100,2.987900


TrainOutput(global_step=500, training_loss=2.341448394775391, metrics={'train_runtime': 737.8859, 'train_samples_per_second': 1.355, 'train_steps_per_second': 0.678, 'total_flos': 130646016000000.0, 'train_loss': 2.341448394775391, 'epoch': 10.0})

In [14]:
# Save the model and tokenizer to directory
trainer.save_model("./fine_tuned_gpt2_demo")
tokenizer.save_pretrained("./fine_tuned_gpt2_demo")

('./fine_tuned_gpt2_demo/tokenizer_config.json',
 './fine_tuned_gpt2_demo/special_tokens_map.json',
 './fine_tuned_gpt2_demo/vocab.json',
 './fine_tuned_gpt2_demo/merges.txt',
 './fine_tuned_gpt2_demo/added_tokens.json')

In [16]:
# Example input text for summarization
input_text = """
Scientists are developing more than 100 coronavirus vaccines using a range of techniques,
some of which are well-established and some of which have never been tested before.
"""

# Detect the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the detected device
model = model.to(device)

# Tokenize the input text and move tensors to the same device
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    max_length=256,
    truncation=True,
    padding=True  # Ensures padding is applied
).to(device)

# Extract input IDs and attention mask from the tokenized inputs
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]

# Ensure the model is in evaluation mode
model.eval()

# Generate the summary with attention mask
outputs = model.generate(
    input_ids=input_ids,
    attention_mask=attention_mask,  # Explicitly provide attention mask
    max_length=50,  # Limit the maximum length of the summary
    min_length=10,  # Minimum length of the summary
    length_penalty=2.0,  # Encourage shorter summaries
    num_beams=4,  # Use beam search for better results
    early_stopping=True  # Stop once the most probable sequence is completed
)

# Decode the generated tokens into text
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated summary
print("Generated Summary:", summary)

Generated Summary: 
Scientists are developing more than 100 coronavirus vaccines using a range of techniques, 
some of which are well-established and some of which have never been tested before.

The coronavirus vaccine, called Covid, is the
